# diet.py

notebook 版本

Solve the classic diet model, showing how to add constraints
to an existing model.

 Nutrition guidelines, based on
 USDA Dietary Guidelines for Americans, 2005
 http://www.health.gov/DietaryGuidelines/dga2005/


### Step 1: Import functions from the gurobipy module

In [1]:
from gurobipy import *

categories, minNutrition, maxNutrition = multidict({
  'calories': [1800, 2200],
  'protein':  [91, GRB.INFINITY],
  'fat':      [0, 65],
  'sodium':   [0, 1779] })
print categories
print minNutrition
print maxNutrition

['protein', 'calories', 'fat', 'sodium']
{'protein': 91, 'calories': 1800, 'fat': 0, 'sodium': 0}
{'protein': 1e+100, 'calories': 2200, 'fat': 65, 'sodium': 1779}


In [2]:
foods, cost = multidict({
  'hamburger': 2.49,
  'chicken':   2.89,
  'hot dog':   1.50,
  'fries':     1.89,
  'macaroni':  2.09,
  'pizza':     1.99,
  'salad':     2.49,
  'milk':      0.89,
  'ice cream': 1.59 })
print foods
print cost

['hamburger', 'salad', 'hot dog', 'fries', 'macaroni', 'chicken', 'milk', 'ice cream', 'pizza']
{'hamburger': 2.49, 'salad': 2.49, 'hot dog': 1.5, 'fries': 1.89, 'macaroni': 2.09, 'chicken': 2.89, 'milk': 0.89, 'ice cream': 1.59, 'pizza': 1.99}


In [3]:
nutritionValues = {
  ('hamburger', 'calories'): 410,
  ('hamburger', 'protein'):  24,
  ('hamburger', 'fat'):      26,
  ('hamburger', 'sodium'):   730,
  ('chicken',   'calories'): 420,
  ('chicken',   'protein'):  32,
  ('chicken',   'fat'):      10,
  ('chicken',   'sodium'):   1190,
  ('hot dog',   'calories'): 560,
  ('hot dog',   'protein'):  20,
  ('hot dog',   'fat'):      32,
  ('hot dog',   'sodium'):   1800,
  ('fries',     'calories'): 380,
  ('fries',     'protein'):  4,
  ('fries',     'fat'):      19,
  ('fries',     'sodium'):   270,
  ('macaroni',  'calories'): 320,
  ('macaroni',  'protein'):  12,
  ('macaroni',  'fat'):      10,
  ('macaroni',  'sodium'):   930,
  ('pizza',     'calories'): 320,
  ('pizza',     'protein'):  15,
  ('pizza',     'fat'):      12,
  ('pizza',     'sodium'):   820,
  ('salad',     'calories'): 320,
  ('salad',     'protein'):  31,
  ('salad',     'fat'):      12,
  ('salad',     'sodium'):   1230,
  ('milk',      'calories'): 100,
  ('milk',      'protein'):  8,
  ('milk',      'fat'):      2.5,
  ('milk',      'sodium'):   125,
  ('ice cream', 'calories'): 330,
  ('ice cream', 'protein'):  8,
  ('ice cream', 'fat'):      10,
  ('ice cream', 'sodium'):   180 }

nutritionValues

{('chicken', 'calories'): 420,
 ('chicken', 'fat'): 10,
 ('chicken', 'protein'): 32,
 ('chicken', 'sodium'): 1190,
 ('fries', 'calories'): 380,
 ('fries', 'fat'): 19,
 ('fries', 'protein'): 4,
 ('fries', 'sodium'): 270,
 ('hamburger', 'calories'): 410,
 ('hamburger', 'fat'): 26,
 ('hamburger', 'protein'): 24,
 ('hamburger', 'sodium'): 730,
 ('hot dog', 'calories'): 560,
 ('hot dog', 'fat'): 32,
 ('hot dog', 'protein'): 20,
 ('hot dog', 'sodium'): 1800,
 ('ice cream', 'calories'): 330,
 ('ice cream', 'fat'): 10,
 ('ice cream', 'protein'): 8,
 ('ice cream', 'sodium'): 180,
 ('macaroni', 'calories'): 320,
 ('macaroni', 'fat'): 10,
 ('macaroni', 'protein'): 12,
 ('macaroni', 'sodium'): 930,
 ('milk', 'calories'): 100,
 ('milk', 'fat'): 2.5,
 ('milk', 'protein'): 8,
 ('milk', 'sodium'): 125,
 ('pizza', 'calories'): 320,
 ('pizza', 'fat'): 12,
 ('pizza', 'protein'): 15,
 ('pizza', 'sodium'): 820,
 ('salad', 'calories'): 320,
 ('salad', 'fat'): 12,
 ('salad', 'protein'): 31,
 ('salad', 'sodiu


### Step 2: Create empty model

In [4]:
m = Model()

### Step 3: Create activitiy variables

In [5]:
# Create decision variables for the nutrition information,
# which we limit via bounds
nutrition = m.addVars(categories, lb=minNutrition, ub=maxNutrition, name="nutrition")

# Create decision variables for the foods to buy
buy = m.addVars(foods, name="buy")

# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding two statements...
#
# nutrition = {}
# for c in categories:
#   nutrition[c] = m.addVar(lb=minNutrition[c], ub=maxNutrition[c], name=c)
#
# buy = {}
# for f in foods:
#   buy[f] = m.addVar(name=f)

### Step 4: Set objective function

In [6]:
# The objective is to minimize the costs
m.setObjective(buy.prod(cost), GRB.MINIMIZE)

### Step 5: Add constraints

In [7]:
m.addConstrs(
    (quicksum(nutritionValues[f,c] * buy[f] for f in foods) == nutrition[c]
     for c in categories), "_")

{'calories': <gurobi.Constr *Awaiting Model Update*>,
 'fat': <gurobi.Constr *Awaiting Model Update*>,
 'protein': <gurobi.Constr *Awaiting Model Update*>,
 'sodium': <gurobi.Constr *Awaiting Model Update*>}

In [8]:
def printSolution():
    if m.status == GRB.Status.OPTIMAL:
        print('\nCost: %g' % m.objVal)
        print('\nBuy:')
        buyx = m.getAttr('x', buy)
        nutritionx = m.getAttr('x', nutrition)
        for f in foods:
            if buy[f].x > 0.0001:
                print('%s %g' % (f, buyx[f]))
        print('\nNutrition:')
        for c in categories:
            print('%s %g' % (c, nutritionx[c]))
    else:
        print('No solution')

### Step 6: Solve model

In [9]:
m.optimize()

Optimize a model with 4 rows, 13 columns and 40 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 3 columns
Presolve time: 0.02s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.601562e+00   0.000000e+00      0s
       4    1.1828861e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.03 seconds
Optimal objective  1.182886111e+01


### Step 8: Print variable values for optimal solution

In [10]:
m.printAttr('X')


    Variable            X 
-------------------------
nutrition[protein]           91 
nutrition[calories]         1800 
nutrition[fat]      59.0559 
nutrition[sodium]         1779 
buy[hamburger]     0.604514 
   buy[milk]      6.97014 
buy[ice cream]      2.59132 


In [11]:
m.getVars()

[<gurobi.Var nutrition[protein] (value 91.0)>,
 <gurobi.Var nutrition[calories] (value 1800.0)>,
 <gurobi.Var nutrition[fat] (value 59.0559027778)>,
 <gurobi.Var nutrition[sodium] (value 1779.0)>,
 <gurobi.Var buy[hamburger] (value 0.604513888889)>,
 <gurobi.Var buy[salad] (value 0.0)>,
 <gurobi.Var buy[hot dog] (value 0.0)>,
 <gurobi.Var buy[fries] (value 0.0)>,
 <gurobi.Var buy[macaroni] (value 0.0)>,
 <gurobi.Var buy[chicken] (value 0.0)>,
 <gurobi.Var buy[milk] (value 6.97013888889)>,
 <gurobi.Var buy[ice cream] (value 2.59131944444)>,
 <gurobi.Var buy[pizza] (value 0.0)>]

In [12]:
m.getConstrs()

[<gurobi.Constr protein>,
 <gurobi.Constr calories>,
 <gurobi.Constr fat>,
 <gurobi.Constr sodium>]

In [14]:
printSolution()


Cost: 11.8289

Buy:
hamburger 0.604514
milk 6.97014
ice cream 2.59132

Nutrition:
protein 91
calories 1800
fat 59.0559
sodium 1779
